This notebook contains the scripts used in the AEI working paper [_A Budget-Neutral Universal Basic Income_](https://www.aei.org/wp-content/uploads/2017/05/UBI-working-paper.pdf).

In [1]:
from taxcalc import *
from functions2 import *
import copy
import pandas as pd
import numpy as np

In [2]:
# Total benefits from cps
cps = pd.read_csv('cps_benefit.csv')
cps['tot_benefits'] = cps['MedicareX'] + cps['MEDICAID'] + cps['SS'] + cps['SSI'] + cps['SNAP'] + cps['VB']
cps_rev = (cps['tot_benefits'] * cps['s006']).sum()

In [3]:
# Total benefits from other programs
other_programs = pd.read_csv('benefitprograms.csv')
other_programs['Cost'] *= 1000000
other_rev = other_programs['Cost'].sum()

In [4]:
# Allocate benefits from other programs to individual
cps['dist_ben'] = cps['MEDICAID'] + cps['SSI'] + cps['SNAP'] + cps['VB']
cps['ratio'] = cps.dist_ben  * cps.s006 / (cps.dist_ben * cps.s006).sum() 
cps['other'] = cps.ratio * other_programs['Cost'].sum() / cps.s006

In [5]:
# Base calculator
recs = Records('puf_benefits.csv', weights='puf_weights_new.csv', adjust_ratios='puf_ratios copy.csv')
calc = Calculator(records=recs, policy=Policy(), verbose=False)
calc.advance_to_year(2014)
calc.calc_all()

In [6]:
# Calculator to measure lost revenue from SS repeal
r_ss = Records('puf_benefits.csv', weights='puf_weights_new.csv', adjust_ratios='puf_ratios copy.csv')
c_ss = Calculator(records=r_ss, policy=Policy(), verbose=False)
c_ss.records.e02400 = np.zeros(len(c_ss.records.e02400))
c_ss.advance_to_year(2014)
c_ss.calc_all()

# Lost Revenue
ss_lostrev = ((c_ss.records.combined - calc.records.combined) * c_ss.records.s006).sum()

In [7]:
cps_storage = copy.deepcopy(cps)

# UBI with original Tax Reform

In [8]:
# Calculator with original tax refrom
recs_reform = Records('puf_benefits.csv', weights='puf_weights_new.csv', adjust_ratios='puf_ratios copy.csv')
pol_reform = Policy()
tax_reform = {
    2014: {
        '_ALD_StudentLoan_hc': [1.0],
        '_ALD_SelfEmploymentTax_hc': [1.0],
        '_ALD_SelfEmp_HealthIns_hc': [1.0],
        '_ALD_KEOGH_SEP_hc': [1.0],
        '_ALD_EarlyWithdraw_hc': [1.0],
        '_ALD_Alimony_hc': [1.0],
        '_ALD_Dependents_hc': [1.0],
        '_ALD_EducatorExpenses_hc': [1.0],
        '_ALD_HSADeduction_hc': [1.0],
        '_ALD_IRAContributions_hc': [1.0],
        '_ALD_DomesticProduction_hc': [1.0],
        '_ALD_Tuition_hc': [1.0],
        '_CR_RetirementSavings_hc': [1.0],
        '_CR_ForeignTax_hc': [1.0],
        '_CR_ResidentialEnergy_hc': [1.0],
        '_CR_GeneralBusiness_hc': [1.0],
        '_CR_MinimumTax_hc': [1.0],
        '_CR_AmOppRefundable_hc': [1.0],
        '_CR_AmOppNonRefundable_hc': [1.0],
        '_CR_SchR_hc': [1.0],
        '_CR_OtherCredits_hc': [1.0],
        '_CR_Education_hc': [1.0],
        '_II_em': [0.0],
        '_STD': [[0.0, 0.0, 0.0, 0.0, 0.0]],
        '_STD_Aged': [[0.0, 0.0, 0.0, 0.0, 0.0]],
        '_ID_Medical_hc': [1.0],
        '_ID_StateLocalTax_hc': [1.0],
        '_ID_RealEstate_hc': [1.0],
        '_ID_InterestPaid_hc': [1.0],
        '_ID_Casualty_hc': [1.0],
        '_ID_Miscellaneous_hc': [1.0],
        '_CDCC_c': [0.0],
        '_CTC_c': [0.0],
        '_EITC_c': [[0.0, 0.0, 0.0, 0.0]],
        '_LLC_Expense_c': [0.0],
        '_ETC_pe_Single': [0.0],
        '_ETC_pe_Married': [0.0]
    }
}
pol_reform.implement_reform(tax_reform)
calc_reform = Calculator(records=recs_reform, policy=pol_reform, verbose=False)
calc_reform.records.e02400 = np.zeros(len(calc_reform.records.e02400))
calc_reform.advance_to_year(2014)
calc_reform.calc_all()

In [9]:
# Revenue from tax reform
tax_rev = ((calc_reform.records.combined - calc.records.combined) * calc_reform.records.s006).sum()

In [10]:
# Total UBI Revenue
revenue = cps_rev + other_rev + ss_lostrev + tax_rev
revenue

3205995011172.418

In [11]:
# Number above and below 18
u18 = (calc_reform.records.nu18 * calc_reform.records.s006).sum()
abv18 = ((calc_reform.records.n1821 + calc_reform.records.n21) * calc_reform.records.s006).sum()

In [12]:
# Find original UBI amounts
ubi18, ubiu18 = ubi_amt(revenue, u18, abv18)
ubi18, ubiu18

(11059.297594986943, 5529.648797493472)

In [13]:
# Find UBI after accounting for UBI tax revenue
diff = 9e99
ubi_tax_rev = 0
prev_ubi_tax_rev = 0
while abs(diff) >= 100:
    ubi18, ubiu18 = ubi_amt(revenue + ubi_tax_rev, u18, abv18)
    diff, ubi_tax_rev = ubi_finder(ubi18, ubiu18, 
                                   tax_reform=tax_reform, revenue=revenue,
                                   calc_reform=calc_reform)
    if diff > 0:
        ubi_tax_rev = prev_ubi_tax_rev * 0.5
    prev_ubi_tax_rev = ubi_tax_rev
ubi18, ubiu18

(13788.339143065874, 6894.169571532937)

In [14]:
# Calculator with UBI and tax reform
recs_ubi1 = Records('puf_benefits.csv', weights='puf_weights_new.csv', adjust_ratios='puf_ratios copy.csv')
pol_ubi1 = Policy()
pol_ubi1.implement_reform(tax_reform)
ubi_ref = {
    2014: {
        '_UBI1': [ubiu18],
        '_UBI2': [ubi18],
        '_UBI3': [ubi18]
    }
}
pol_ubi1.implement_reform(ubi_ref)
calc_ubi1 = Calculator(records=recs_ubi1, policy=pol_ubi1, verbose=False)
calc_ubi1.records.e02400 = np.zeros(len(calc_ubi1.records.e02400))
calc_ubi1.advance_to_year(2014)
calc_ubi1.calc_all()

In [15]:
# Get MTR's
mtrs = calc_ubi1.mtr()

In [16]:
pd.options.display.float_format = '{:,.2f}'.format

## For all Tax Units

In [17]:
table_data1 = prep_table_data(calc=calc_ubi1, calc_base=calc, mtrs=mtrs, bins='income')

In [18]:
avg_ben, avg_ben_mult = cps_avg_ben(cps_storage, other_programs, group='all', bins='income')
table(table_data1, avg_ben, avg_ben_mult)

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg UBI Per Person,Avg MTR - Payroll,Avg MTR - Individual Income,Avg MTR - Combined,Avg Tax Change,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Benefits Change - Welfare Adj,Avg Combined Change,Avg Combined Change - Welfare Adj
0,0,76.51,1.93,"12,122",14.2%,14.5%,28.7%,"4,958","22,483","-24,481","-20,390","-6,956","-2,865"
1,10,16.16,1.64,"12,542",14.2%,15.5%,29.7%,"6,714","20,158","-9,848","-7,795","3,596","5,649"
2,20,13.74,1.74,"12,228",14.2%,19.7%,33.9%,"7,640","20,968","-9,195","-7,281","4,134","6,048"
3,30,11.09,1.76,"12,583",14.2%,22.0%,36.2%,"8,169","21,312","-7,812","-6,286","5,330","6,856"
4,40,8.54,1.80,"12,668",14.2%,22.7%,36.9%,"8,814","21,918","-8,502","-6,807","4,602","6,297"
5,50,15.35,2.01,"12,684",14.2%,23.4%,37.5%,"10,607","24,490","-7,297","-5,963","6,586","7,919"
6,75,9.02,2.44,"12,409",14.2%,24.2%,38.4%,"13,914","28,970","-5,679","-4,818","9,376","10,238"
7,100,13.72,2.77,"12,184",10.9%,26.6%,37.4%,"17,875","32,325","-5,619","-4,819","8,831","9,631"
8,200,4.32,3.09,"11,960",6.1%,33.7%,39.8%,"28,425","35,307","-5,368","-4,717","1,514","2,164"
9,"1,001",0.17,3.15,"12,024",4.1%,39.1%,43.2%,"148,695","36,026","-8,281","-7,743","-120,951","-120,413"


## Tax Units w/ Someone above 65

In [19]:
table_data2 = prep_table_data(calc=calc_ubi1, calc_base=calc, mtrs=mtrs, group='65 or over', bins='income')
#somehow s006 has to be the first variable in table? why?!!

In [20]:
avg_ben, avg_ben_mult = cps_avg_ben(cps_storage, other_programs, group='65 or over', bins='income')
table(table_data2, avg_ben, avg_ben_mult)

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg UBI Per Person,Avg MTR - Payroll,Avg MTR - Individual Income,Avg MTR - Combined,Avg Tax Change,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Benefits Change - Welfare Adj,Avg Combined Change,Avg Combined Change - Welfare Adj
0,0,29.33,1.73,"13,437",14.2%,16.4%,30.6%,"4,718","22,947","-46,090","-40,690","-27,861","-22,461"
1,10,3.42,1.95,"13,507",14.2%,16.4%,30.6%,"7,838","26,128","-34,960","-31,159","-16,670","-12,868"
2,20,2.73,1.99,"13,548",14.2%,20.6%,34.8%,"8,500","26,742","-34,863","-31,478","-16,621","-13,236"
3,30,0.93,2.00,"13,518",14.2%,20.5%,34.7%,"7,677","26,748","-33,526","-29,992","-14,456","-10,922"
4,40,0.54,2.00,"13,423",14.2%,21.7%,35.8%,"7,195","26,428","-36,390","-32,703","-17,157","-13,471"
5,50,0.87,2.03,"13,524",14.2%,23.5%,37.6%,"8,163","27,146","-31,962","-29,027","-12,979","-10,044"
6,75,0.44,2.00,"13,542",14.2%,24.3%,38.5%,"9,227","26,859","-31,104","-28,637","-13,471","-11,005"
7,100,0.52,2.09,"13,530",9.5%,27.0%,36.5%,"12,378","28,047","-33,951","-31,687","-18,282","-16,018"
8,200,0.17,2.00,"13,621",6.9%,34.5%,41.4%,"23,433","27,022","-29,537","-27,590","-25,948","-24,002"
9,"1,001",0.01,1.90,"13,788",5.6%,38.4%,44.0%,"137,770","26,222","-22,774","-22,464","-134,323","-134,013"


## Tax Units w/out Someone Over 65

In [21]:
table_data3 = prep_table_data(calc=calc_ubi1, calc_base=calc, mtrs=mtrs, group='under 65', bins='income')

In [22]:
avg_ben, avg_ben_mult = cps_avg_ben(cps_storage, other_programs, group='under 65', bins='income')
table(table_data3, avg_ben, avg_ben_mult)

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg UBI Per Person,Avg MTR - Payroll,Avg MTR - Individual Income,Avg MTR - Combined,Avg Tax Change,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Benefits Change - Welfare Adj,Avg Combined Change,Avg Combined Change - Welfare Adj
0,0,47.19,2.05,"11,305",14.2%,13.3%,27.4%,"5,107","22,195","-13,757","-10,316","3,331","6,773"
1,10,12.74,1.56,"12,284",14.2%,15.2%,29.4%,"6,413","18,558","-6,833","-4,990","5,312","7,155"
2,20,11.01,1.68,"11,900",14.2%,19.5%,33.7%,"7,426","19,535","-6,321","-4,572","5,788","7,537"
3,30,10.16,1.74,"12,498",14.2%,22.1%,36.3%,"8,214","20,816","-4,802","-3,511","7,799","9,090"
4,40,8.01,1.79,"12,617",14.2%,22.8%,36.9%,"8,923","21,616","-5,175","-3,718","7,518","8,975"
5,50,14.47,2.01,"12,633",14.2%,23.4%,37.5%,"10,755","24,330","-4,376","-3,232","9,199","10,342"
6,75,8.58,2.46,"12,351",14.2%,24.2%,38.4%,"14,153","29,077","-3,171","-2,468","11,753","12,456"
7,100,13.21,2.80,"12,132",10.9%,26.6%,37.5%,"18,090","32,493","-3,003","-2,338","11,399","12,064"
8,200,4.15,3.14,"11,893",6.0%,33.7%,39.7%,"28,627","35,642","-2,664","-2,158","4,351","4,856"
9,"1,001",0.16,3.25,"11,892",4.0%,39.1%,43.1%,"149,515","36,762","-6,497","-5,931","-119,251","-118,685"


# UBI + Original Tax Reform + New Provisions

New Provisions: No AMT, personal income and pass through rates: <$50K (single) / <$100K (joint) - 10%, >$50K (single) / >$100K (joint) - 50%

In [23]:
# Calculator with second reform policy
recs_reform2 = Records('puf_benefits.csv', weights='puf_weights_new.csv', adjust_ratios='puf_ratios copy.csv')
pol_reform2 = Policy()
pol_reform2.implement_reform(tax_reform)
tax_reform2 = {
    2014: {
        '_ALD_StudentLoan_hc': [1.0],
        '_ALD_SelfEmploymentTax_hc': [1.0],
        '_ALD_SelfEmp_HealthIns_hc': [1.0],
        '_ALD_KEOGH_SEP_hc': [1.0],
        '_ALD_EarlyWithdraw_hc': [1.0],
        '_ALD_Alimony_hc': [1.0],
        '_ALD_Dependents_hc': [1.0],
        '_ALD_EducatorExpenses_hc': [1.0],
        '_ALD_HSADeduction_hc': [1.0],
        '_ALD_IRAContributions_hc': [1.0],
        '_ALD_DomesticProduction_hc': [1.0],
        '_ALD_Tuition_hc': [1.0],
        '_CR_RetirementSavings_hc': [1.0],
        '_CR_ForeignTax_hc': [1.0],
        '_CR_ResidentialEnergy_hc': [1.0],
        '_CR_GeneralBusiness_hc': [1.0],
        '_CR_MinimumTax_hc': [1.0],
        '_CR_AmOppRefundable_hc': [1.0],
        '_CR_AmOppNonRefundable_hc': [1.0],
        '_CR_SchR_hc': [1.0],
        '_CR_OtherCredits_hc': [1.0],
        '_CR_Education_hc': [1.0],
        '_II_em': [0.0],
        '_STD': [[0.0, 0.0, 0.0, 0.0, 0.0]],
        '_STD_Aged': [[0.0, 0.0, 0.0, 0.0, 0.0]],
        '_ID_Medical_hc': [1.0],
        '_ID_StateLocalTax_hc': [1.0],
        '_ID_RealEstate_hc': [1.0],
        '_ID_InterestPaid_hc': [1.0],
        '_ID_Casualty_hc': [1.0],
        '_ID_Miscellaneous_hc': [1.0],
        '_CDCC_c': [0.0],
        '_CTC_c': [0.0],
        '_EITC_c': [[0.0, 0.0, 0.0, 0.0]],
        '_LLC_Expense_c': [0.0],
        '_ETC_pe_Single': [0.0],
        '_ETC_pe_Married': [0.0],
        '_II_rt2': [.10],
        '_II_rt3': [.10],
        '_II_rt4': [.10],
        '_II_rt5': [.10],
        '_II_rt6': [.10],
        '_II_rt7': [.50],
        '_II_brk1': [[50000, 100000, 50000, 50000, 100000]],
        '_II_brk2': [[50000, 100000, 50000, 50000, 100000]],
        '_II_brk3': [[50000, 100000, 50000, 50000, 100000]],
        '_II_brk4': [[50000, 100000, 50000, 50000, 100000]],
        '_II_brk5': [[50000, 100000, 50000, 50000, 100000]],
        '_II_brk6': [[50000, 100000, 50000, 50000, 100000]],
        '_PT_rt2': [.10],
        '_PT_rt3': [.10],
        '_PT_rt4': [.10],
        '_PT_rt5': [.10],
        '_PT_rt6': [.10],
        '_PT_rt7': [.50],
        '_PT_brk1': [[50000, 100000, 50000, 50000, 100000]],
        '_PT_brk2': [[50000, 100000, 50000, 50000, 100000]],
        '_PT_brk3': [[50000, 100000, 50000, 50000, 100000]],
        '_PT_brk4': [[50000, 100000, 50000, 50000, 100000]],
        '_PT_brk5': [[50000, 100000, 50000, 50000, 100000]],
        '_PT_brk6': [[50000, 100000, 50000, 50000, 100000]],
        '_AMT_rt1': [0.0],
        '_AMT_rt2': [0.0]
    }
}
pol_reform2.implement_reform(tax_reform2)
calc_reform2 = Calculator(records=recs_reform2, policy=pol_reform2, verbose=False)
calc_reform2.records.e02400 = np.zeros(len(calc_reform2.records.e02400))
calc_reform2.advance_to_year(2014)
calc_reform2.calc_all()

In [24]:
# Revenue from tax reform
tax_rev2 = ((calc_reform2.records.combined - calc.records.combined) * calc_reform2.records.s006).sum()

In [25]:
revenue2 = cps_rev + other_rev + ss_lostrev + tax_rev2
revenue2

3525615655995.964

In [26]:
# Find original UBI amounts
ubi18, ubiu18 = ubi_amt(revenue2, u18, abv18)
ubi18, ubiu18

(12161.850723200503, 6080.925361600252)

In [27]:
# Find UBI after accounting for UBI tax revenue
diff = 9e99
ubi_tax_rev = 0
prev_ubi_tax_rev = 0
while abs(diff) >= 300:
    ubi18, ubiu18 = ubi_amt(revenue2 + ubi_tax_rev, u18, abv18)
    diff, ubi_tax_rev = ubi_finder(ubi18, ubiu18, 
                                   tax_reform=tax_reform2, revenue=revenue2,
                                   calc_reform=calc_reform2)
    if diff > 0:
        ubi_tax_rev = prev_ubi_tax_rev * 0.5
    prev_ubi_tax_rev = ubi_tax_rev
    print diff
ubi18, ubiu18

-7.92004018102e+11
-2.1808784331e+11
-62813016455.9
-18387484308.9
-5413251291.1
-1596045234.54
-470793013.282
-138886183.643
-40971757.2188
-12086919.7422
-3565717.92773
-1051908.74805
-310317.999023
-91545.4482422
-27006.2011719
-7966.87988281
-2348.921875
-692.581054688
-204.416015625


(15952.820515200001, 7976.4102576000005)

In [28]:
# Calculator with UBI and tax reform
recs_ubi2 = Records('puf_benefits.csv', weights='puf_weights_new.csv', adjust_ratios='puf_ratios copy.csv')
pol_ubi2 = Policy()
pol_ubi2.implement_reform(tax_reform)
ubi_ref2 = {
    2014: {
        '_UBI1': [ubiu18],
        '_UBI2': [ubi18],
        '_UBI3': [ubi18]
    }
}
pol_ubi2.implement_reform(ubi_ref2)
calc_ubi2 = Calculator(records=recs_ubi2, policy=pol_ubi2, verbose=False)
calc_ubi2.records.e02400 = np.zeros(len(calc_ubi2.records.e02400))
calc_ubi2.advance_to_year(2014)
calc_ubi2.calc_all()

# Get MTR's
# try using baseline MTR
mtrs2 = calc.mtr()

## For all Tax Units

In [29]:
table_data4 = prep_table_data(calc=calc_ubi2, calc_base=calc, mtrs=mtrs2, bins='income')

In [30]:
avg_ben, avg_ben_mult = cps_avg_ben(cps_storage, other_programs, group='all', bins='income')
table(table_data4, avg_ben, avg_ben_mult)

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg UBI Per Person,Avg MTR - Payroll,Avg MTR - Individual Income,Avg MTR - Combined,Avg Tax Change,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Benefits Change - Welfare Adj,Avg Combined Change,Avg Combined Change - Welfare Adj
0,0,76.51,1.93,"14,025",14.2%,1.0%,15.2%,"5,633","26,013","-24,481","-20,390","-4,101",-10
1,10,16.16,1.64,"14,511",14.2%,6.0%,20.2%,"7,277","23,323","-9,848","-7,795","6,197","8,251"
2,20,13.74,1.74,"14,147",14.2%,15.9%,30.1%,"8,366","24,260","-9,195","-7,281","6,700","8,613"
3,30,11.09,1.76,"14,558",14.2%,17.9%,32.1%,"8,960","24,657","-7,812","-6,286","7,885","9,411"
4,40,8.54,1.80,"14,656",14.2%,17.0%,31.2%,"9,660","25,359","-8,502","-6,807","7,197","8,891"
5,50,15.35,2.01,"14,675",14.2%,18.4%,32.6%,"11,579","28,335","-7,297","-5,963","9,459","10,792"
6,75,9.02,2.44,"14,357",14.2%,19.0%,33.2%,"15,094","33,518","-5,679","-4,818","12,744","13,606"
7,100,13.72,2.77,"14,097",10.9%,25.0%,35.9%,"19,312","37,400","-5,619","-4,819","12,468","13,268"
8,200,4.32,3.09,"13,838",6.1%,32.4%,38.4%,"30,352","40,849","-5,368","-4,717","5,130","5,780"
9,"1,001",0.17,3.15,"13,912",4.1%,39.2%,43.3%,"150,941","41,682","-8,281","-7,743","-117,541","-117,003"


## Tax Units with Someone Over 65

In [31]:
table_data5 = prep_table_data(calc=calc_ubi2, calc_base=calc, mtrs=mtrs2, group='65 or over', bins='income')

In [32]:
avg_ben, avg_ben_mult = cps_avg_ben(cps_storage, other_programs, group='65 or over', bins='income')
table(table_data5, avg_ben, avg_ben_mult)

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg UBI Per Person,Avg MTR - Payroll,Avg MTR - Individual Income,Avg MTR - Combined,Avg Tax Change,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Benefits Change - Welfare Adj,Avg Combined Change,Avg Combined Change - Welfare Adj
0,0,29.33,1.73,"15,546",14.2%,5.4%,19.6%,"5,369","26,549","-46,090","-40,690","-24,910","-19,510"
1,10,3.42,1.95,"15,627",14.2%,8.7%,22.9%,"8,598","30,230","-34,960","-31,159","-13,328","-9,527"
2,20,2.73,1.99,"15,675",14.2%,19.0%,33.2%,"9,475","30,940","-34,863","-31,478","-13,397","-10,013"
3,30,0.93,2.00,"15,640",14.2%,21.5%,35.7%,"8,622","30,946","-33,526","-29,992","-11,202","-7,668"
4,40,0.54,2.00,"15,530",14.2%,20.5%,34.7%,"8,190","30,576","-36,390","-32,703","-14,004","-10,317"
5,50,0.87,2.03,"15,647",14.2%,19.2%,33.4%,"9,246","31,407","-31,962","-29,027","-9,801","-6,866"
6,75,0.44,2.00,"15,668",14.2%,22.3%,36.4%,"10,332","31,075","-31,104","-28,637","-10,360","-7,894"
7,100,0.52,2.09,"15,654",9.5%,26.2%,35.7%,"13,639","32,450","-33,951","-31,687","-15,141","-12,876"
8,200,0.17,2.00,"15,759",6.9%,33.4%,40.3%,"24,952","31,264","-29,537","-27,590","-23,225","-21,279"
9,"1,001",0.01,1.90,"15,953",5.6%,38.8%,44.4%,"139,394","30,338","-22,774","-22,464","-131,831","-131,521"


## Tax Units w/out Someone Over 65

In [33]:
table_data6 = prep_table_data(calc=calc_ubi2, calc_base=calc, mtrs=mtrs2, group='under 65', bins='income')

In [34]:
avg_ben, avg_ben_mult = cps_avg_ben(cps_storage, other_programs, group='under 65', bins='income')
table(table_data6, avg_ben, avg_ben_mult)

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg UBI Per Person,Avg MTR - Payroll,Avg MTR - Individual Income,Avg MTR - Combined,Avg Tax Change,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Benefits Change - Welfare Adj,Avg Combined Change,Avg Combined Change - Welfare Adj
0,0,47.19,2.05,"13,079",14.2%,-1.8%,12.4%,"5,797","25,680","-13,757","-10,316","6,125","9,567"
1,10,12.74,1.56,"14,212",14.2%,5.3%,19.4%,"6,923","21,471","-6,833","-4,990","7,715","9,559"
2,20,11.01,1.68,"13,768",14.2%,15.1%,29.3%,"8,090","22,602","-6,321","-4,572","8,190","9,939"
3,30,10.16,1.74,"14,460",14.2%,17.6%,31.8%,"8,991","24,083","-4,802","-3,511","10,290","11,581"
4,40,8.01,1.79,"14,598",14.2%,16.8%,30.9%,"9,759","25,009","-5,175","-3,718","10,075","11,532"
5,50,14.47,2.01,"14,616",14.2%,18.3%,32.5%,"11,720","28,149","-4,376","-3,232","12,053","13,196"
6,75,8.58,2.46,"14,290",14.2%,18.9%,33.0%,"15,337","33,642","-3,171","-2,468","15,134","15,837"
7,100,13.21,2.80,"14,036",10.9%,25.0%,35.9%,"19,534","37,594","-3,003","-2,338","15,056","15,721"
8,200,4.15,3.14,"13,760",6.0%,32.3%,38.3%,"30,570","41,237","-2,664","-2,158","8,003","8,508"
9,"1,001",0.16,3.25,"13,759",4.0%,39.3%,43.2%,"151,807","42,532","-6,497","-5,931","-115,772","-115,206"
